# Run Predictions Evaluation - Markov - Extreme Travelers 0.1.categ_v1

In [1]:
import os
os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())


working dir /home/tales/dev/master/mdc_analysis


In [2]:
import json
import urllib.request
from itertools import groupby

from src.ml.ctw import MyCTW
from src.dao import csv_dao
from src.dao import objects_dao
from src.entity.stop_region import StopRegionGroup, sr_row_to_stop_region
from src.exceptions import exceptions
from src.exceptions.exceptions import NoCategoryMatched
from src.experiments.ctw_eval import evaluation_ctw_single_partition_light_mem
from src.experiments.markov_eval import evaluation_markov_k_fold_light_mem, evaluation_markov_all_users_vs_one_light_mem, evaluation_markov_single_partition_light_mem, evaluation_markov_cluster_light_mem
from src.taxonomy.category_mapping import tags_to_categ
from src.exceptions.exceptions import NoCategoryMatched, NotValidTypes

from src.data_processment.input_data_version.py import InputDataManager

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Loading Tags Sequence Data

In [4]:
def get_users_tags_sequence():
    url = "http://127.0.0.1:5000/stop_regions_group"
    response = urllib.request.urlopen(url)
    return json.loads(response.read())

def load_users_raw_tags_sequence():
    with open('outputs/users_tags_sequence.json') as json_file:
        return json.load(json_file)
    
tags_sequence_30_m = objects_dao.load_users_tags_sequence(sr_stay_time_above_h=0.5)

users_tags_sequence_original = tags_sequence_30_m["original"]
users_tags_sequence_30_m = tags_sequence_30_m["filtered"]
users_tags_sequence_15_m = objects_dao.load_users_tags_sequence(sr_stay_time_above_h=0.25)["filtered"]

print(len(users_tags_sequence_original.keys()))

Loading Stop Region Group data
Building Stop Region Group sequence
Loading Stop Region Group data
Building Stop Region Group sequence
163


## Mapping Tags to Category
version: <b>0.1.categ_v1</b>
(categ versrion 1, with equal consecutive categories agglutinated)

In [6]:
users_categ_sequence_original, users_categ_sequence_elements_original = tags_to_categ(users_tags_sequence_original)
users_categ_sequence_30_m, users_categ_sequence_elements_30_m = tags_to_categ(users_tags_sequence_30_m)
users_categ_sequence_15_m, users_categ_sequence_elements_15_m = tags_to_categ(users_tags_sequence_15_m)

n: 1 user_id: 6015
n: 2 user_id: 6086
n: 3 user_id: 6014
n: 4 user_id: 6057
n: 5 user_id: 6181
n: 6 user_id: 6069
n: 7 user_id: 5985
n: 8 user_id: 5970
n: 9 user_id: 6062
n: 10 user_id: 6042
n: 11 user_id: 5966
n: 12 user_id: 6067
n: 13 user_id: 5949
n: 14 user_id: 5938
n: 15 user_id: 5937
n: 16 user_id: 5980
n: 17 user_id: 6075
n: 18 user_id: 6028
n: 19 user_id: 5968
n: 20 user_id: 6036
n: 21 user_id: 6023
n: 22 user_id: 6078
n: 23 user_id: 6031
n: 24 user_id: 5927
n: 25 user_id: 5942
n: 26 user_id: 6030
n: 27 user_id: 6087
n: 28 user_id: 6035
n: 29 user_id: 6029
n: 30 user_id: 6171
n: 31 user_id: 6199
n: 32 user_id: 5977
n: 33 user_id: 5973
n: 34 user_id: 6197
n: 35 user_id: 5928
n: 36 user_id: 6003
n: 37 user_id: 6178
n: 38 user_id: 6040
n: 39 user_id: 6179
n: 40 user_id: 5955
n: 41 user_id: 5962
n: 42 user_id: 6180
n: 43 user_id: 5956
n: 44 user_id: 6176
n: 45 user_id: 6182
n: 46 user_id: 5924
n: 47 user_id: 6070
n: 48 user_id: 5944
n: 49 user_id: 6072
n: 50 user_id: 6051
n: 51 use

n: 80 user_id: 6100
n: 81 user_id: 6079
n: 82 user_id: 6000
n: 83 user_id: 5988
n: 84 user_id: 6109
n: 85 user_id: 5925
n: 86 user_id: 6047
n: 87 user_id: 5950
n: 88 user_id: 6190
n: 89 user_id: 5974
n: 90 user_id: 6004
n: 91 user_id: 6053
n: 92 user_id: 5959
n: 93 user_id: 5978
n: 94 user_id: 6104
n: 95 user_id: 6041
n: 96 user_id: 6096
n: 97 user_id: 6094
n: 98 user_id: 6001
n: 99 user_id: 6174
n: 100 user_id: 6076
n: 101 user_id: 6059
n: 102 user_id: 6017
n: 103 user_id: 6188
n: 104 user_id: 6071
n: 105 user_id: 6034
n: 106 user_id: 6085
n: 107 user_id: 6032
n: 108 user_id: 6172
n: 109 user_id: 6166
n: 110 user_id: 6192
n: 111 user_id: 5947
n: 112 user_id: 6012
n: 113 user_id: 6063
n: 114 user_id: 6082
n: 115 user_id: 5961
n: 116 user_id: 6168
n: 117 user_id: 6026
n: 118 user_id: 6020
n: 119 user_id: 5972
n: 120 user_id: 6007
n: 121 user_id: 6183
n: 122 user_id: 6198
n: 123 user_id: 6056
n: 124 user_id: 6187
n: 125 user_id: 5945
n: 126 user_id: 6189
n: 127 user_id: 6045
n: 128 user_

In [ ]:
len(users_categ_sequence_original)

# Extreme Travelers

#### All users as the cluster

In [ ]:
n=0
for user_id in list(users_categ_sequence_original.keys()):
    n += 1
    print("n:", n, "-", "user_id:", user_id)
    
    try:
        evaluation_markov_cluster_light_mem(cluster=users_categ_sequence_original, 
                                            test_user_id=user_id, 
                                            input_data_version="0.1.categ_v1", 
                                            cluster_version="all_vs_one",
                                            dir_name="clusters_all_vs_one")
    except KeyError:
        print("Skipping user: {}".format(user_id))
    
    try:
        evaluation_markov_cluster_light_mem(cluster=users_categ_sequence_30_m, 
                                            test_user_id=user_id, 
                                            input_data_version="0.1.categ_v1_filtered_30min", 
                                            cluster_version="all_vs_one",
                                            dir_name="clusters_all_vs_one")
    except KeyError:
        print("Skipping user: {}".format(user_id))
        
    try:
        evaluation_markov_cluster_light_mem(cluster=users_categ_sequence_15_m, 
                                            test_user_id=user_id, 
                                            input_data_version="0.1.categ_v1_filtered_15min", 
                                            cluster_version="all_vs_one",
                                            dir_name="clusters_all_vs_one")
    except KeyError:
        print("Skipping user: {}".format(user_id))

#### All users as one cluster <font color="red">for each user of extreme travelers groups</font>

In [ ]:
eb_group = ['5936', '5938', '5959', '5962', '6020', '6023', '6035', '6043', '6103', '6104', '6168', '6171', '6194']

n=0
for user_id in eb_group:
    n += 1
    print("n:", n, "-", "user_id:", user_id)
    
    try:
        evaluation_markov_cluster_light_mem(cluster=users_categ_sequence_original, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1", 
                                        cluster_version="all_vs_early_birds",
                                        dir_name="clusters_all_vs_early_birds")
    except KeyError:
        print("Skipping user: {}".format(user_id))
    
    try:
        evaluation_markov_cluster_light_mem(cluster=users_categ_sequence_30_m, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1_filtered_30min", 
                                        cluster_version="all_vs_early_birds",
                                        dir_name="clusters_all_vs_early_birds")
    except KeyError:
        print("Skipping user: {}".format(user_id))
        
    try:
        evaluation_markov_cluster_light_mem(cluster=users_categ_sequence_15_m, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1_filtered_15min", 
                                        cluster_version="all_vs_early_birds",
                                        dir_name="clusters_all_vs_early_birds")
    except KeyError:
        print("Skipping user: {}".format(user_id))

In [ ]:
no_group = ["6103","6190","6074","6102","6078","5951","6183","6100","6077","6056","6181","6182","6198","6172","6062","5987"]

n=0
for user_id in no_group:
    n += 1
    print("n:", n, "-", "user_id:", user_id)
    try:
        evaluation_markov_cluster_light_mem(cluster=users_categ_sequence_original, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1", 
                                        cluster_version="all_vs_night_owls",
                                        dir_name="clusters_all_vs_night_owls")
    except KeyError:
        print("Skipping user: {}".format(user_id))
    
    try:
        evaluation_markov_cluster_light_mem(cluster=users_categ_sequence_30_m, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1_filtered_30min", 
                                        cluster_version="all_vs_night_owls",
                                        dir_name="clusters_all_vs_night_owls")
    except KeyError:
        print("Skipping user: {}".format(user_id))
        
    try:
        evaluation_markov_cluster_light_mem(cluster=users_categ_sequence_15_m, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1_filtered_15min", 
                                        cluster_version="all_vs_night_owls",
                                        dir_name="clusters_all_vs_night_owls")
    except KeyError:
        print("Skipping user: {}".format(user_id))

In [ ]:
ti_group = ['5928', '5935', '5938', '5940', '5959', '5965', '5979', '5988', '5989', '6014', '6075', '6085', '6090', '6106', '6177', '6188', '6189', '6194']

n=0
for user_id in ti_group:
    n += 1
    print("n:", n, "-", "user_id:", user_id)
    
    try:
        evaluation_markov_cluster_light_mem(cluster=users_categ_sequence_original, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1", 
                                        cluster_version="all_vs_tireless_its",
                                        dir_name="clusters_all_vs_tireless_its")
    except KeyError:
        print("Skipping user: {}".format(user_id))
    
    try:
        evaluation_markov_cluster_light_mem(cluster=users_categ_sequence_30_m, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1_filtered_30min", 
                                        cluster_version="all_vs_tireless_its",
                                        dir_name="clusters_all_vs_tireless_its")
    except KeyError:
        print("Skipping user: {}".format(user_id))
        
    try:
        evaluation_markov_cluster_light_mem(cluster=users_categ_sequence_15_m, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1_filtered_15min", 
                                        cluster_version="all_vs_tireless_its",
                                        dir_name="clusters_all_vs_tireless_its")
    except KeyError:
        print("Skipping user: {}".format(user_id))

#### Extreme travelers, each group trained to test each user on it.
Obs: train partition excluding the test user

In [ ]:
eb_cluster_original = {}
eb_cluster_30_m = {}
eb_cluster_15_m = {}

for eb_user_id in eb_group:
    try:
        eb_cluster_original[eb_user_id] = users_categ_sequence_original[eb_user_id]
        eb_cluster_30_m[eb_user_id] = users_categ_sequence_30_m[eb_user_id]
        eb_cluster_15_m[eb_user_id] = users_categ_sequence_15_m[eb_user_id]
        
    except KeyError:
        print("Skipping user: {}".format(user_id))
    
n=0
for user_id in eb_group:
    n += 1
    print("n:", n, "-", "user_id:", user_id)
    try:
        evaluation_markov_cluster_light_mem(cluster=eb_cluster_original, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1", 
                                        cluster_version="cluster_early_birds",
                                        dir_name="clusters_early_birds")
    except KeyError:
        print("Skipping user: {}".format(user_id))
    
    try:
        evaluation_markov_cluster_light_mem(cluster=eb_cluster_30_m, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1_filtered_30min", 
                                        cluster_version="cluster_early_birds",
                                        dir_name="clusters_early_birds")
    except KeyError:
        print("Skipping user: {}".format(user_id))
        
    try:
        evaluation_markov_cluster_light_mem(cluster=eb_cluster_15_m, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1_filtered_15min", 
                                        cluster_version="cluster_early_birds",
                                        dir_name="clusters_early_birds")
    except KeyError:
        print("Skipping user: {}".format(user_id))

In [ ]:
no_cluster_original = {}
no_cluster_30_m = {}
no_cluster_15_m = {}

for no_user_id in no_group:
    try:
        no_cluster_original[no_user_id] = users_categ_sequence_original[no_user_id]
        no_cluster_30_m[no_user_id] = users_categ_sequence_30_m[no_user_id]
        no_cluster_15_m[no_user_id] = users_categ_sequence_15_m[no_user_id]
        
    except KeyError:
        print("Skipping user: {}".format(user_id))
    
n=0
for user_id in no_group:
    n += 1
    print("n:", n, "-", "user_id:", user_id)
    try:
        evaluation_markov_cluster_light_mem(cluster=no_cluster_original, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1", 
                                        cluster_version="cluster_night_owls",
                                        dir_name="clusters_night_owls")
    except KeyError:
        print("Skipping user: {}".format(user_id))
    
    try:
        evaluation_markov_cluster_light_mem(cluster=no_cluster_30_m, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1_filtered_30min", 
                                        cluster_version="cluster_night_owls",
                                        dir_name="clusters_night_owls")
    except KeyError:
        print("Skipping user: {}".format(user_id))
        
    try:
        evaluation_markov_cluster_light_mem(cluster=no_cluster_15_m, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1_filtered_15min", 
                                        cluster_version="cluster_night_owls",
                                        dir_name="clusters_night_owls")
    except KeyError:
        print("Skipping user: {}".format(user_id))

In [ ]:
ti_cluster_original = {}
ti_cluster_30_m = {}
ti_cluster_15_m = {}

for ti_user_id in ti_group:
    try:
        ti_cluster_original[ti_user_id] = users_categ_sequence_original[ti_user_id]
        ti_cluster_30_m[ti_user_id] = users_categ_sequence_30_m[ti_user_id]
        ti_cluster_15_m[ti_user_id] = users_categ_sequence_15_m[ti_user_id]
        
    except KeyError:
        print("Skipping user [ti group]: {}".format(user_id))
    
n=0
for user_id in ti_group:
    n += 1
    print("n:", n, "-", "user_id:", user_id)
    try:
        evaluation_markov_cluster_light_mem(cluster=ti_cluster_original, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1", 
                                        cluster_version="cluster_tireless_its",
                                        dir_name="clusters_tireless_its")
    except KeyError:
        print("Skipping user [original]: {}".format(user_id))
    
    try:        
        evaluation_markov_cluster_light_mem(cluster=ti_cluster_30_m, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1_filtered_30min", 
                                        cluster_version="cluster_tireless_its",
                                        dir_name="clusters_tireless_its")
    except KeyError:
        print("Skipping user [30m]: {}".format(user_id))
        
    try:        
        evaluation_markov_cluster_light_mem(cluster=ti_cluster_15_m, 
                                        test_user_id=user_id, 
                                        input_data_version="0.1.categ_v1_filtered_15min", 
                                        cluster_version="cluster_tireless_its",
                                        dir_name="clusters_tireless_its")
    except KeyError:
        print("Skipping user [15m]: {}".format(user_id))

#### Extreme travelers - each user by themselves

In [ ]:
n=0
for user_id in eb_group + no_group + ti_group:
    n += 1
    print("n:", n, "-", "user_id:", user_id)
    try:
        evaluation_markov_cluster_light_mem(cluster={user_id: users_categ_sequence_original[user_id], 
                                                     "clone": users_categ_sequence_original[user_id]}, 
                                            test_user_id=user_id, 
                                            input_data_version="0.1.categ_v1", 
                                            cluster_version="extreme_trav_each_one_by_itself",
                                            dir_name="extreme_trav_each_one_by_itself")
    except KeyError:
        print("Skipping user: {}".format(user_id))
    
    try:
        evaluation_markov_cluster_light_mem(cluster={user_id: users_categ_sequence_30_m[user_id], 
                                                     "clone": users_categ_sequence_30_m[user_id]}, 
                                            test_user_id=user_id, 
                                            input_data_version="0.1.categ_v1_filtered_30min", 
                                            cluster_version="extreme_trav_each_one_by_itself",
                                            dir_name="extreme_trav_each_one_by_itself")
    except KeyError:
        print("Skipping user: {}".format(user_id))
        
    try:
        evaluation_markov_cluster_light_mem(cluster={user_id: users_categ_sequence_15_m[user_id], 
                                                     "clone": users_categ_sequence_15_m[user_id]}, 
                                            test_user_id=user_id, 
                                            input_data_version="0.1.categ_v1_filtered_15min", 
                                            cluster_version="extreme_trav_each_one_by_itself",
                                            dir_name="extreme_trav_each_one_by_itself")
    except KeyError:
        print("Skipping user: {}".format(user_id))

In [ ]:
n=0
for user_id in list(users_categ_sequence_original.keys()):
    n += 1
    print("n:", n, "-", "user_id:", user_id)
    
    try:
        evaluation_markov_cluster_light_mem(cluster={user_id: users_categ_sequence_original[user_id], 
                                                     "clone": users_categ_sequence_original[user_id]}, 
                                            test_user_id=user_id, 
                                            input_data_version="0.1.categ_v1", 
                                            cluster_version="each_one_by_itself",
                                            dir_name="each_one_by_itself")
    except KeyError:
        print("Skipping user: {}".format(user_id))
    
    try:
        evaluation_markov_cluster_light_mem(cluster={user_id: users_categ_sequence_30_m[user_id], 
                                                     "clone": users_categ_sequence_30_m[user_id]}, 
                                            test_user_id=user_id, 
                                            input_data_version="0.1.categ_v1_filtered_30min", 
                                            cluster_version="each_one_by_itself",
                                            dir_name="each_one_by_itself")
    except KeyError:
        print("Skipping user: {}".format(user_id))
        
    try:
        evaluation_markov_cluster_light_mem(cluster={user_id: users_categ_sequence_15_m[user_id], 
                                                     "clone": users_categ_sequence_15_m[user_id]}, 
                                            test_user_id=user_id, 
                                            input_data_version="0.1.categ_v1_filtered_15min", 
                                            cluster_version="each_one_by_itself",
                                            dir_name="each_one_by_itself")
    except KeyError:
        print("Skipping user: {}".format(user_id))

In [ ]:
len(users_categ_sequence_original)

##### CREATE DIRs

mkdir /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_all_vs_one/<br> 
<br> 
mkdir /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_all_vs_early_birds/<br> 
mkdir /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_all_vs_night_owls/<br> 
mkdir /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_all_vs_tireless_its/<br> 
<br> 
mkdir /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_early_birds/<br> 
mkdir /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_night_owls/<br> 
mkdir /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_tireless_its/<br> 
<br> 
mkdir /home/tales/dev/master/mdc_analysis/outputs/experiments/each_one_by_itself/<br> 
<br> 
mkdir /home/tales/dev/master/mdc_analysis/outputs/experiments/extreme_trav_each_one_by_itself/<br> 
<br> 
mkdir /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters

##### COPY

cp /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_all_vs_one/* /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters<br> 
<br> 
cp /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_all_vs_early_birds/* /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters<br> 
cp /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_all_vs_night_owls/* /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters<br> 
cp /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_all_vs_tireless_its/* /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters<br> 
<br> 
cp /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_early_birds/* /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters<br> 
cp /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_night_owls/* /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters<br> 
cp /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_tireless_its/* /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters<br> 
<br> 
cp /home/tales/dev/master/mdc_analysis/outputs/experiments/each_one_by_itself/* /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters<br> 
<br> 
cp /home/tales/dev/master/mdc_analysis/outputs/experiments/extreme_trav_each_one_by_itself/* /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters

##### DELETE

rm /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_all_vs_one/ <br> 
rm /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_all_vs_early_birds/ <br> 
rm /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_all_vs_night_owls/ <br> 
rm /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_all_vs_tireless_its/ <br> 
<br> 
rm /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_early_birds/ <br> 
rm /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_night_owls/ <br> 
rm /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters_tireless_its/ <br> 
<br> 
rm /home/tales/dev/master/mdc_analysis/outputs/experiments/each_one_by_itself/ <br> 
<br> 
rm /home/tales/dev/master/mdc_analysis/outputs/experiments/extreme_trav_each_one_by_itself/ <br> 
<br> 
rm /home/tales/dev/master/mdc_analysis/outputs/experiments/clusters/